In [1]:
import pandas as pd
df = pd.read_csv('../Data/Clean Data/K2 P&C Clean.csv') 
df  

,pl_orbper,pl_orbsmax,pl_orbeccen,pl_orbincl,pl_imppar,pl_ratdor,pl_radj,pl_masse,pl_massj,pl_bmasse,...,sy_vmag,sy_bmag,sy_kepmag,sy_tmag,sy_snum,sy_pnum,sy_dist,sy_pm,sy_plx,Encoded_Labels
0,41.688644,0.104828,0.120751,89.525540,0.438159,54.721028,0.210139,121.83621,0.383344,130.182165,...,10.849,11.765,11.040,10.4021,1,1,179.4610,63.053943,5.54366,1
1,41.685500,0.241000,0.000000,89.550000,0.438159,55.800000,0.198947,16.30000,0.051285,16.300000,...,10.849,11.765,11.040,10.4021,1,1,179.4610,63.053943,5.54366,1
2,41.685500,0.104828,0.000000,89.550000,0.438159,55.800000,0.230000,22.24810,0.070000,22.248100,...,10.849,11.765,11.040,10.4021,1,1,179.4610,63.053943,5.54366,1
3,2.301830,0.104828,0.120751,87.601528,0.420000,11.800000,0.099900,121.83621,0.383344,130.182165,...,11.727,12.737,11.363,10.7606,1,0,97.1795,73.413760,10.26160,0
4,2.302368,0.104828,0.120751,87.443840,0.438159,12.619867,0.120000,121.83621,0.383344,130.182165,...,11.727,12.737,11.363,10.7606,1,0,97.1795,73.413760,10.26160,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,2.655676,0.104828,0.120751,87.601528,0.438159,7.834000,0.990000,121.83621,0.383344,130.182165,...,10.720,11.428,10.247,10.1500,2,1,141.8870,78.021443,7.01970,1
4000,6.001270,0.057060,0.410000,87.601528,0.650000,22.451693,0.182265,6.26000,0.019696,6.260000,...,10.270,11.276,9.945,9.2493,1,1,44.5260,667.845078,22.43000,1
4001,6.001180,0.057100,0.120751,87.601528,0.387000,22.451693,0.181104,121.83621,0.383344,130.182165,...,10.270,11.276,9.945,9.2493,1,1,44.5260,667.845078,22.43000,1
4002,6.001270,0.057120,0.409000,89.870000,0.438159,22.451693,0.182265,6.27000,0.019728,6.270000,...,10.270,11.276,9.945,9.2493,1,1,44.5260,667.845078,22.43000,1


In [5]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [6]:
X=df.drop('Encoded_Labels',axis=1) 

In [7]:
y=df['Encoded_Labels']

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
model = XGBClassifier(
    n_estimators=300,       # number of trees
    max_depth=6,            # tree depth
    learning_rate=0.05,     # step size shrinkage
    subsample=0.8,          # random row sampling
    colsample_bytree=0.8,   # random column sampling
    objective='multi:softmax',
    num_class=3,            # 3 classes
    eval_metric='mlogloss',
    random_state=42
)

In [11]:
model.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None, num_class=3, ...)

In [12]:
y_pred = model.predict(X_test) 

In [13]:
print(classification_report(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94       295
           1       1.00      0.98      0.99       450
           2       0.70      0.93      0.80        54
           3       0.67      1.00      0.80         2

    accuracy                           0.96       801
   macro avg       0.83      0.96      0.88       801
weighted avg       0.96      0.96      0.96       801

[[273   0  21   1]
 [  7 443   0   0]
 [  4   0  50   0]
 [  0   0   0   2]]


In [14]:
df['Encoded_Labels'].unique()


array([1, 0, 2, 3], dtype=int64)

In [15]:
df_subset = X_train.iloc[:, :100]

# Save the subset to a CSV file
df_subset.to_csv('K2_Test.csv', index=False)

In [16]:
import pickle

# Save the model
with open("model_k2.pkl", "wb") as f:
    pickle.dump(model, f)